In [1]:
import sys
sys.path.insert(0, '..')

from api import API
import pandas as pd

from marthas_dashboard.views import (
    tools)

from bokeh.plotting import (figure, show)
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
api = API()

## Room Compare

In [3]:
searches = {
    'building': '4',
    'date': '2017-08-18',
    'timestamp': '00:00:00'}

search_results = tools.get_room_comparison_results(searches)
search_results

,name_room,pointid_valve,valve,pointid_temp1 (RM),temp1 (RM),pointid_temp2 (RMT),temp2 (RMT)
0,102,608,12.0,609,73.42,610,73.42
1,112,572,3.0,573,70.98,574,70.98
2,121,589,9.0,590,70.72,591,70.72
3,202,622,15.0,623,69.70,624,71.11
4,203,640,9.0,641,67.62,642,67.62
5,206,631,9.0,632,69.59,633,69.59
6,207,662,9.0,663,76.87,664,76.87
7,212,668,3.0,669,72.15,670,72.15
8,213,682,9.0,683,71.65,684,71.65
9,215,699,9.0,700,69.28,701,69.28


## Room Inspector

In [4]:
from datetime import (datetime, timedelta)
TIME_FMT = "%Y-%m-%d %H:%M:%S"

from tools.room_compare import hacky_tagging, merge_tables

In [5]:
searches = {
    'building': '4',
    'date': '2017-08-18',
    'timestamp': '00:00:00',
    'room': '102',
    'temp1': '73.42',
    'temp2': '73.42',
    'valve': '12.0',
}

tools.get_room_inspector_results(searches)

,datetime,temp1 (RM),temp2 (RMT),valve
0,2017-08-18 00:00:00,73.42,73.42,12.0
1,2017-08-18 00:15:00,73.40,73.40,12.0
2,2017-08-18 00:30:00,73.40,73.40,12.0
3,2017-08-18 00:45:00,73.39,73.39,12.0
4,2017-08-18 01:00:00,73.37,73.37,12.0
5,2017-08-18 01:15:00,73.38,73.38,12.0
6,2017-08-18 01:30:00,73.37,73.37,12.0
7,2017-08-18 01:45:00,73.39,73.39,12.0
8,2017-08-18 02:00:00,73.36,73.36,12.0
9,2017-08-18 02:15:00,73.35,73.35,12.0


In [6]:
# From url args (searches): get "ts" (date + timestamp); building_id; room_name
ts = datetime.strptime(' '.join([searches['date'], searches['timestamp']]), TIME_FMT)
building_id = searches['building']
room_name = searches['room']

# Compute "start" (12 hours earlier) and "stop" (12 hours later) than timestamp
start = (ts - timedelta(hours=12)).strftime(TIME_FMT)
end = (ts + timedelta(hours=12)).strftime(TIME_FMT)

# Make api calls (maybe this should be a single SQL call)
points = api.building_points(building_id)
rooms = api.building_rooms(building_id)
vals = api.building_values_in_range(building_id, start, end)

In [7]:
df = merge_tables(points, rooms, vals)

In [8]:
df = hacky_tagging(df)

In [15]:
df['datetime'] = pd.to_datetime(df.date + ' ' + df.time)

In [18]:
df

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time,tag,datetime
0,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 00:00:00,150.82,float,DEG F,2017-08-18,00:00,none,2017-08-18 00:00:00
1,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 00:15:00,149.42,float,DEG F,2017-08-18,00:15,none,2017-08-18 00:15:00
2,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 00:30:00,150.36,float,DEG F,2017-08-18,00:30,none,2017-08-18 00:30:00
3,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 00:45:00,150.66,float,DEG F,2017-08-18,00:45,none,2017-08-18 00:45:00
4,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 01:00:00,148.75,float,DEG F,2017-08-18,01:00,none,2017-08-18 01:00:00
5,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 01:15:00,151.03,float,DEG F,2017-08-18,01:15,none,2017-08-18 01:15:00
6,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 01:30:00,149.39,float,DEG F,2017-08-18,01:30,none,2017-08-18 01:30:00
7,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 01:45:00,150.30,float,DEG F,2017-08-18,01:45,none,2017-08-18 01:45:00
8,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 02:00:00,150.58,float,DEG F,2017-08-18,02:00,none,2017-08-18 02:00:00
9,LAB HOT H2O TEMP,NaN,511,BI.LAB.HW.TEMP,2,13,6,4,6,Hulings_Dummy_Room,...,511,BI.LAB.HW.TEMP,2017-08-18 02:15:00,148.83,float,DEG F,2017-08-18,02:15,none,2017-08-18 02:15:00


## Load / Merge Data

In [4]:
d = api.buildings()
bldings = list(d.values())

In [5]:
# Jons stuff

# import sys
# sys.path.insert(0, '..')

# from marthas_dashboard.views import (
#     get_room_comparison_results,
#     get_room_names_from_point_names)

# searches = dict()
# searches['building'] = '4'
# searches['date'] = '2017-08-18'
# searches['timestamp'] = '00:00:00'

# search_results = get_room_comparison_results(searches)
# search_results.head()

# sr2 = get_room_names_from_point_names(searches, vals)
# sr2.head()

In [6]:
bldings_with_rooms = ['Hulings']

rm = bldings_with_rooms[0]
print(rm)
bld = api.building(rm).id
timestamp = '2017-08-18 00:45:00'

# Load some data
rooms = api.building_rooms(bld)
points = api.building_points(bld)
vals = api.building_values_at_time(bld, timestamp)

Hulings


In [7]:
d = {'date':'2017-08-18', 'timestamp':'00:00:00', 'room':'102'}
TIME_FMT = "%Y-%m-%d %H:%M:%S"

from datetime import datetime, timedelta

time_string = ' '.join([d['date'], d['timestamp']])
ts = datetime.strptime(time_string, TIME_FMT)

start = (ts - timedelta(hours=12)).strftime(TIME_FMT)
end = (ts + timedelta(hours=12)).strftime(TIME_FMT)

In [8]:
points = api.building_points('4')
rooms = api.building_rooms('4')
vals = api.building_values_in_range('4', start, end)

rooms = rooms[rooms['name'] == d['room']]

# Merge df together
df = pd.merge(
    points, rooms,
    left_on='roomid', right_on='id',
    suffixes=('_point', '_room'))

df= pd.merge(df, vals, left_on='id_point', right_on='pointid',)

In [9]:
df['tag'] = 'none'

# Tag 'valve' based on description
df_valve = df[df['description'].str.find('VALVE') > 0]
df.loc[df_valve.index, 'tag'] = 'valve'

# Tag 'temp' based on point name
df_temp = df[df['name_point'].str.find('.RM') > 0]
df.loc[df_temp.index, 'tag'] = 'temp1 (RM)'

df_temp = df[df['name_point'].str.find('.RMT') > 0]
df.loc[df_temp.index, 'tag'] = 'temp2 (RMT)'

df['datetime'] = pd.to_datetime(df.date + ' ' + df.time)

view_df = df.query('tag != "none"')

In [10]:
tags = ['valve', 'temp1 (RM)', 'temp2 (RMT)']

view_df = (view_df.pivot(index='datetime', columns='tag', values='pointvalue')
           .reset_index().rename_axis(None, axis=1))

In [11]:
view_df['x'] = view_df['datetime']

view_df['y'] = view_df['valve']

In [13]:
from bokeh.layouts import column, row
from bokeh.models import (
    CustomJS, ColumnDataSource, RadioButtonGroup, Select, Slider)
from bokeh.plotting import (Figure, show)
from bokeh.io import output_notebook
# from bokeh.models.widgets import RadioButtonGroup
output_notebook()

Loading BokehJS ...

In [14]:
view_df

,datetime,temp1 (RM),temp2 (RMT),valve,x,y
0,2017-08-18 00:00:00,73.42,73.42,12.0,2017-08-18 00:00:00,12.0
1,2017-08-18 00:15:00,73.40,73.40,12.0,2017-08-18 00:15:00,12.0
2,2017-08-18 00:30:00,73.40,73.40,12.0,2017-08-18 00:30:00,12.0
3,2017-08-18 00:45:00,73.39,73.39,12.0,2017-08-18 00:45:00,12.0
4,2017-08-18 01:00:00,73.37,73.37,12.0,2017-08-18 01:00:00,12.0
5,2017-08-18 01:15:00,73.38,73.38,12.0,2017-08-18 01:15:00,12.0
6,2017-08-18 01:30:00,73.37,73.37,12.0,2017-08-18 01:30:00,12.0
7,2017-08-18 01:45:00,73.39,73.39,12.0,2017-08-18 01:45:00,12.0
8,2017-08-18 02:00:00,73.36,73.36,12.0,2017-08-18 02:00:00,12.0
9,2017-08-18 02:15:00,73.35,73.35,12.0,2017-08-18 02:15:00,12.0


In [21]:
source = ColumnDataSource(data=view_df)

plot = Figure(plot_width=400, plot_height=400, x_axis_type='datetime')

plot.line('x', 'y' , source=source, line_width=2)

def callback(source=source, window=None):
    data = source.data
    attr = cb_obj.active
    x = data['datetime']
    y = data['temp1 (RM)']
    for i in range(len(x)):
        y[i] = window.Math.pow(x[i], 2)
    source.change.emit()

radio_button_group = RadioButtonGroup(
    labels=tags, active=0, callback=CustomJS.from_py_func(callback))

# sel = Select(options=tags, callback=CustomJS.from_py_func(callback))
# sel.js_on_change('value', callback)

# layout = column(radio_button_group, plot)
layout = column(radio_button_group, plot)

show(layout)

In [22]:
from bokeh.embed import components

In [24]:
s, div = components(layout)

In [27]:
s

'\n<script type="text/javascript">\n  (function() {\n    var fn = function() {\n      Bokeh.safely(function() {\n        (function(root) {\n          function embed_document(root) {\n            \n          var docs_json = \'{"2287491f-592d-4c03-9c6f-a3296ef6bfec":{"roots":{"references":[{"attributes":{"formatter":{"id":"51199deb-5f37-47f2-aeac-003b86d326b0","type":"BasicTickFormatter"},"plot":{"id":"71680bda-3f50-41ce-851e-f250e75a3837","subtype":"Figure","type":"Plot"},"ticker":{"id":"53d19e26-baca-4a32-aa8b-b91e9d17fdf8","type":"BasicTicker"}},"id":"14d7645a-8640-40e0-b214-4c81ea261a0f","type":"LinearAxis"},{"attributes":{"children":[{"id":"179b7970-adfb-4e04-8cad-22832be5d442","type":"RadioButtonGroup"}]},"id":"09e1971b-410a-4c91-9afa-3123b0d73186","type":"WidgetBox"},{"attributes":{"dimension":1,"plot":{"id":"71680bda-3f50-41ce-851e-f250e75a3837","subtype":"Figure","type":"Plot"},"ticker":{"id":"53d19e26-baca-4a32-aa8b-b91e9d17fdf8","type":"BasicTicker"}},"id":"da479400-a674-4543-

In [50]:
source.data['temp1 (RM)']

array([73.42, 73.4 , 73.4 , 73.39, 73.37, 73.38, 73.37, 73.39, 73.36,
       73.35, 73.36, 73.34, 73.32, 73.31, 73.29, 73.29, 73.29, 73.27,
       73.27, 73.27, 73.27, 73.26, 73.21, 73.25, 73.22, 73.23, 73.23,
       73.25, 73.22, 73.2 , 73.26, 73.31, 73.28, 73.06, 72.33, 71.9 ,
       71.72, 71.61, 71.57, 71.56, 71.54, 71.53, 71.57, 71.61, 71.64,
       71.67, 71.81, 71.75])

In [47]:
y

array([73.42, 73.4 , 73.4 , 73.39, 73.37, 73.38, 73.37, 73.39, 73.36,
       73.35, 73.36, 73.34, 73.32, 73.31, 73.29, 73.29, 73.29, 73.27,
       73.27, 73.27, 73.27, 73.26, 73.21, 73.25, 73.22, 73.23, 73.23,
       73.25, 73.22, 73.2 , 73.26, 73.31, 73.28, 73.06, 72.33, 71.9 ,
       71.72, 71.61, 71.57, 71.56, 71.54, 71.53, 71.57, 71.61, 71.64,
       71.67, 71.81, 71.75])

In [19]:
x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(a=x, b=y))

plot = Figure(plot_width=400, plot_height=400)
plot.line('a', 'b', source=source, line_width=3, line_alpha=0.6)

def callback(source=source, window=None):
    data = source.data
    f = cb_obj.value
    x, y = data['x'], data['y']
    for i in range(len(x)):
        y[i] = window.Math.pow(x[i], f)
    source.change.emit()

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power",
                callback=CustomJS.from_py_func(callback))

layout = column(slider, plot)

show(layout)

In [71]:
# display(rooms.head())
# display(points.head())
# display(vals.head())

In [72]:
# Remove room 6, the scary dummy room
rooms = rooms[rooms['name'].str.find('_Dummy_') < 0]

In [74]:
# Merge df together
df = pd.merge(
    points, rooms,
    left_on='roomid', right_on='id',
    suffixes=('_point', '_room'))

df= pd.merge(df, vals, left_on='id_point', right_on='pointid',)

In [75]:
df.head(n=10)

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time
0,112 RH VALVE,NaN,572,HU.R112.RHV,2,1,18,4,18,112,...,1,floatPSI2,572,HU.R112.RHV,2017-08-18 00:45:00,3.00,float,PSI,2017-08-18,00:45
1,112 TEMP,NaN,573,HU.R112.RM,2,13,18,4,18,112,...,13,floatDEG F2,573,HU.R112.RM,2017-08-18 00:45:00,70.92,float,DEG F,2017-08-18,00:45
2,112 TEMP,NaN,574,HU.R112.RMT,2,13,18,4,18,112,...,13,floatDEG F2,574,HU.R112.RMT,2017-08-18 00:45:00,70.92,float,DEG F,2017-08-18,00:45
3,112 SUP CFM,NaN,575,HU.R112.SACFM,2,699,18,4,18,112,...,699,floatC CFM2,575,HU.R112.SACFM,2017-08-18 00:45:00,0.00,float,C CFM,2017-08-18,00:45
4,112 TOT SUP,NaN,576,HU.R112.TSUP,2,47,18,4,18,112,...,47,floatCFM2,576,HU.R112.TSUP,2017-08-18 00:45:00,0.00,float,CFM,2017-08-18,00:45
5,121 EXH CFM,NaN,587,HU.R121.ECFM,2,47,19,4,19,121,...,47,floatCFM2,587,HU.R121.ECFM,2017-08-18 00:45:00,127.87,float,CFM,2017-08-18,00:45
6,121 EXH CFM,NaN,588,HU.R121.EXCFM,2,699,19,4,19,121,...,699,floatC CFM2,588,HU.R121.EXCFM,2017-08-18 00:45:00,1.28,float,C CFM,2017-08-18,00:45
7,121 RH VALVE,NaN,589,HU.R121.RHV,2,1,19,4,19,121,...,1,floatPSI2,589,HU.R121.RHV,2017-08-18 00:45:00,9.00,float,PSI,2017-08-18,00:45
8,121 TEMP,NaN,590,HU.R121.RM,2,13,19,4,19,121,...,13,floatDEG F2,590,HU.R121.RM,2017-08-18 00:45:00,70.61,float,DEG F,2017-08-18,00:45
9,121 TEMP,NaN,591,HU.R121.RMT,2,13,19,4,19,121,...,13,floatDEG F2,591,HU.R121.RMT,2017-08-18 00:45:00,70.61,float,DEG F,2017-08-18,00:45


## Attempt to tag specific points

In [12]:
df['tag'] = 'none'

# Tag 'valve' based on description
df_valve = df[df['description'].str.find('VALVE') > 0]
df.loc[df_valve.index, 'tag'] = 'valve'

# Tag 'temp' based on point name
df_temp = df[df['name_point'].str.find('.RM') > 0]
df.loc[df_temp.index, 'tag'] = 'temp1 (RM)'

df_temp = df[df['name_point'].str.find('.RMT') > 0]
df.loc[df_temp.index, 'tag'] = 'temp2 (RMT)'

In [13]:
# View DF where tag is not "none
view_df = df.query('tag != "none"')

# Pivot 
view_df = (view_df.pivot(index='name_room', columns='tag',
                        values='pointvalue')
           .reset_index().rename_axis(None, axis=1))

view_df

,name_room,temp1 (RM),temp2 (RMT),valve
0,102,73.39,73.39,12.0
1,112,70.92,70.92,3.0
2,121,70.61,70.61,9.0
3,202,69.60,71.01,15.0
4,203,67.65,67.65,9.0
5,206,69.10,69.10,9.0
6,207,76.93,76.93,9.0
7,212,72.18,72.18,3.0
8,213,71.60,71.60,9.0
9,215,69.11,69.11,9.0


In [15]:
for idx, row in view_df.iterrows():
    print(row.name_room)

NameError: name 'view_df' is not defined

In [13]:
view_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name_room</th>\n      <th>temp1 (RM)</th>\n      <th>temp2 (RMT)</th>\n      <th>valve</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>102</td>\n      <td>73.39</td>\n      <td>73.39</td>\n      <td>12.0</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>112</td>\n      <td>70.92</td>\n      <td>70.92</td>\n      <td>3.0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>121</td>\n      <td>70.61</td>\n      <td>70.61</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>202</td>\n      <td>69.60</td>\n      <td>71.01</td>\n      <td>15.0</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>203</td>\n      <td>67.65</td>\n      <td>67.65</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>206</td>\n      <td>69.10</td>\n      <td>69.10</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th

In [ ]:
def get_rooms_points(buildings):
    """maps building ids to their points and rooms,
    ie {4:{'rooms':{5}}}"""
    result = {}
    for building_id, name in buildings.items():
        building_data = {
            'rooms': map_rooms(api.building_rooms(building_id)),
            'points': map_points(api.building_points(building_id))
        }
        result[building_id] = building_data
    return result

def map_points(points):
    """Take a pandas df returned from building_points
    and turn it into something easier to use on the frontend
    """
    results = {}
    for index, row in points.iterrows():
        results[row['id']] = row['name'] + '- ' + row['description']
    return results


def map_rooms(rooms):
    """Similar to the map_points function but for rooms"""
    results = {}
    for index, row in rooms.iterrows():
        results[row['id']] = row['name'].replace("_", " ")
    return results